# Lab 6 - ERC 20 (Part II)

## 1. Setup

### Step 1: Copy content of lab-5 into lab-6

We will make use of the config, .env and packages installed from previous lab.


In [ ]:
cp -rp ~/github/suss/fin579/lab-5/. ~/github/suss/fin579/lab-6
cd ~/github/suss/fin579/lab-6

## 2. Lab 6a - MintableERC20 Token

Create a new contract called MintableERC20 contract based on ERC20 contract created in the lab-4. The MintableERC20 tokens will be minted using the mint() function.

### Step 1. Create Mintable Contract

Create `MintableErc20.sol` in `contracts` directory.

```js
pragma solidity 0.8.8;
import './ERC20.sol';

contract MintableERC20 is ERC20 {

}
```

### Step 2.  Create constructor.

When creating a constructor for a derived contract, both the derived contract's constructor and the base contract's constructor must be initialised together. The base contract's constructor can be invoked using the base contract's name.

```js
constructor(string memory name_, string memory symbol_)
    ERC20(name_,symbol_,0,address(0))
{
}

```

Because the tokens will be minted and sent to a recipient's address, we will set the initial `totalSupply` and `recipient` of the base contract to `0` and `address(0)` for `totalSupply` and `recipient` respectively.

### Step 3.  Create the mint() function.

*   The mint() function will increase the totalSupply by `amount` and increase balance for the address `to` by `amount`.
*   The mint() function is declared as **virtual** so that the derived contract can override and replace its implementation.

```js
function mint(address to, uint256 amount) virtual
public
{
    _totalSupply += amount;
    _balances[to] += amount;
    emit Transfer(address(0), to, amount);
}
```

### Completed Code

```js
pragma solidity 0.8.8;
import "./ERC20.sol";

contract MintableERC20 is ERC20 {
    constructor(
        string memory name_,
        string memory symbol_
    ) ERC20(name_, symbol_, 0, address(0)) {}

    function mint(address to, uint256 amount) public virtual {
        _totalSupply += amount;
        _balances[to] += amount;
        emit Transfer(address(0), to, amount);
    }
}

```

In [ ]:
hh compile

## 3. Lab 6b - Test Mintable Contract

### Step 1.  Create `testMintableErc20.js` in `test` directory.

    We will deploy MintableERC20 before each test and call it `m20`.

    ```js
    const { expect } = require("chai");

    describe("Test MintableERC20", () => {
        let accounts;
        let m20;

        beforeEach(async () => {
            accounts = await ethers.getSigners();
            const factory = await ethers.getContractFactory("MintableERC20");
            m20 = await factory.deploy("m20", "m20");
        });
    });
    ```

### Step 2.  Create a test case `Test MintableERC20`.

    The test case will mint 100 units of m20 to `accounts[1]` by calling the `MintableERC20.mint()` function. The test is successful when the balance of m20 for `accounts[1]` is increased by 100.

    ```js
    it("Should mint 100 unit of m20 with 10000 wei of ETH", async () => {
        const before = await m20.balanceOf(accounts[1].address);

        const response = await m20.mint(accounts[1].address, 100);
        await response.wait();

        // Assert
        const after = await m20.balanceOf(accounts[1].address);
        expect(before.add(100).toNumber()).equals(after.toNumber());
    });
    ```

### Step 3.  Run test


In [ ]:
hh test test/testMintableErc20.js

## 4. Lab 6c - Ownable Contract

The MintableERC20 contract allows anyone to `mint()` tokens for anyone else. In this session, we will create a contract to manage the access to the mint() function using a `owner` address. Since ownership is a rather generic function, we will create a standalone `Ownership` contract. This contract can subsequently be used by the TutorialContract via multiple inheritance.

### Step 1. Create Ownable Contract

Create `Ownable.sol` in `contracts` directory.

```sol
pragma solidity 0.8.8;

contract Ownable {

}
```

### Step 2. Create the state variable `owner`.

```sol
address public owner;
```

### Step 3.  Create the constructor

This is to allow contract to be assigned an owner at the point of creation.

```sol
constructor(address _owner) {
    owner=_owner;
}
```

### Step 4.  Create the transferOwnership function

This is to allow the ownership of the contract to be transferred to another address.

```sol
event OwnershipTransferred(address indexed previousOwner, address indexed newOwner);
function transferOwnership(address newOwner)
public
virtual
{
    require(owner == msg.sender, "Ownable: caller is not the owner");
    require(newOwner != address(0), "Ownable: new owner is the zero address");
    address oldOwner = owner;
    owner = newOwner;
    emit OwnershipTransferred(oldOwner, newOwner);
}
```

Since the expression that checks for ownership `require(owner == msg.sender, "Ownable: caller is not the owner")` can be commonly used, we can extract it into a function to make it reusable. Or we can take advantage of Solidity's **function modifier** feature as described below.

### Step 5.  Create `onlyOwner` modifier

Add the following modifier to the contract. A modifier is a syntactic sugar that lets you attach code to the beginning or end of a function. The basic use case is used for validation purpose like in this case, used to check if the caller is contract owner. The advantage is that onlyOwner can be added as a modifier to any function instead of repeating the same if then statement.

```sol
modifier onlyOwner() {
    require(owner == msg.sender, "Ownable: caller is not the owner");
    _;
}
```

In the above code snippet, the ownership check will be called each time the modifier is added to a function.

### Step 6.  Update the transferOwnership function.

Protect the function using `onlyOwner` modifier.

```sol
event OwnershipTransferred(address indexed previousOwner, address indexed newOwner);
function transferOwnership(address newOwner)
public
onlyOwner
virtual
{
    require(newOwner != address(0), "Ownable: new owner is the zero address");
    address oldOwner = owner;
    owner = newOwner;
    emit OwnershipTransferred(oldOwner, newOwner);
}
```

The line of code containing the ownership check is removed from the function body and the `onlyOwner` modifier is added to the function declaration.

### Completed Code

```sol
pragma solidity 0.8.8;

contract Ownable {
    address public owner;

    constructor(address _owner) {
        owner=_owner;
    }

    modifier onlyOwner() {
        require(owner == msg.sender, "Ownable: caller is not the owner");
        _;
    }

    event OwnershipTransferred(address indexed previousOwner, address indexed newOwner);

    function transferOwnership(address newOwner) public virtual onlyOwner {
        require(newOwner != address(0), "Ownable: new owner is the zero address");
        address oldOwner = owner;
        owner = newOwner;
        emit OwnershipTransferred(oldOwner, newOwner);
    }

}
```


## 5. Lab 6d - TutorialERC20 Token

In this session, we will use Multiple Contract Inheritance to derive the TutorialERC20 Contract. This is the final ERC20 contract that will incorporate the characteristics of ERC20, MintableERC20 and Ownable contracts.

### Step 1. Create TutorialERC20 Contract

Create the file `TutorialERC20.sol` in the `contracts` directory.

```sol
pragma solidity 0.8.8;
import './MintableERC20.sol';
import './Ownable.sol';

contract TutorialERC20 is MintableERC20, Ownable {

```

### Step 2. Create the constructor.
Since the contract is inheriting from MintableERC20 and Ownable, the constructor for both base contracts needs to be called.

```sol
    constructor(string memory name_, string memory symbol_, address owner_)
        MintableERC20(name_,symbol_)
        Ownable(owner_)
    {

    }
```

### Step 3. Override the mint() function.

Override the mint() function from MintableERC20 and give access to owner only using the onlyOwner modifier.

```js
function mint(address to, uint256 amount) override
onlyOwner
public
{
    _totalSupply += amount;
    _balances[to] += amount;
    emit Transfer(address(0), to, amount);
}
```
    
### Completed Code

```js
pragma solidity 0.8.8;
import "./MintableERC20.sol";
import "./Ownable.sol";

contract TutorialERC20 is MintableERC20, Ownable {
    constructor(
        string memory name_,
        string memory symbol_,
        address owner_
    ) MintableERC20(name_, symbol_) Ownable(owner_) {}

    function mint(address to, uint256 amount) public override onlyOwner {
        _totalSupply += amount;
        _balances[to] += amount;
        emit Transfer(address(0), to, amount);
    }
}
```


In [ ]:
hh compile

## 6. Test TutorialERC20 Token

### Step 1. Create testTutorialERC20 test script

Create the test `testTutorialERC20.js`.

    ```js
    const { expect } = require("chai");
    const { ethers } = require("hardhat");

    describe("Test TutorialERC20", () => {
        let accounts;
        let tut;

        beforeEach(async () => {
            accounts = await ethers.getSigners();
            let factory = await ethers.getContractFactory("TutorialERC20");
            tut = await factory.deploy(
                "TUT",
                "Tutorial ERC20 Token",
                accounts[0].address
            );
        });
    ```

The scope must cover 3 cases:

1. Owner can mint token.
2. Non-Owner cannot mint token.
3. Can mint after transferring ownership

    ##### Owner can mint token

    ```js
    it("Account 0 can mint token", async () => {
        const before = await tut.balanceOf(accounts[1].address);
        const response = await tut.mint(accounts[1].address, 100);
        await response.wait();
        // Assert
        const after = await tut.balanceOf(accounts[1].address);
        expect(before.add(100).toNumber()).equals(after.toNumber());
    });
    ```

    ##### Non-Owner cannot mint token

    ```js
    it("Account 1 cannot mint token", async () => {
        // Alternatively, install chai-as-promised and use expect().eventually syntax.
        try {
            await tut.connect(accounts[1]).mint(accounts[1].address, 100);
        } catch (err) {
            expect(err.toString().includes("Ownable: caller is not the owner"));
        }
    });
    ```

    ##### Can mint after transferring ownership

    ```js
    it("can mint after transferOwnership", async () => {
        const before = await tut.balanceOf(accounts[1].address);
        await tut.transferOwnership(accounts[1].address);
        const response = await tut
            .connect(accounts[1])
            .mint(accounts[1].address, 100);
        await response.wait();
        // Assert
        const after = await tut.balanceOf(accounts[1].address);
        expect(before.add(100).toNumber()).equals(after.toNumber());
    });
    ```

### Step 5. Run tests.

```js
const { expect } = require("chai");
const { ethers } = require("hardhat");

describe("Test TutorialERC20", () => {
    let accounts;
    let tut;

    beforeEach(async () => {
        accounts = await ethers.getSigners();
        let factory = await ethers.getContractFactory("TutorialERC20");
        tut = await factory.deploy(
            "TUT",
            "Tutorial ERC20 Token",
            accounts[0].address
        );
    });

    it("Account 0 can mint token", async () => {
        const before = await tut.balanceOf(accounts[1].address);
        const response = await tut.mint(accounts[1].address, 100);
        await response.wait();
        // Assert
        const after = await tut.balanceOf(accounts[1].address);
        expect(before.add(100).toNumber()).equals(after.toNumber());
    });

    it("Account 1 cannot mint token", async () => {
        // Alternatively, install chai-as-promised and use expect().eventually syntax.
        try {
            await tut.connect(accounts[1]).mint(accounts[1].address, 100);
        } catch (err) {
            expect(err.toString().includes("Ownable: caller is not the owner"));
        }
    });

    it("can mint after transferOwnership", async () => {
        const before = await tut.balanceOf(accounts[1].address);
        await tut.transferOwnership(accounts[1].address);
        const response = await tut
            .connect(accounts[1])
            .mint(accounts[1].address, 100);
        await response.wait();
        // Assert
        const after = await tut.balanceOf(accounts[1].address);
        expect(before.add(100).toNumber()).equals(after.toNumber());
    });
});

```

In [ ]:
hh test test/testTutorialERC20

## 7. Lab 6e - Deploy to Testnet

### Step 1. Create a deployment script deploy-TUT.js

    ```js
    const { ethers } = require("hardhat");

    (async () => {
        let account = await ethers.getSigner();
        let factory = await ethers.getContractFactory("TutorialERC20");
        let deployed = await factory.deploy("TUT", "TUT", account.address);
        console.log("TUT address:", deployed.address);
    })();
    ```

### Step 2. Deploy to testnet

3. Check etherscan

    ```sh
    https://rinkeby.etherscan.io/address/...
    ```


In [ ]:
hh run deploy-TUT.js --network goerli

### Step 3. Check etherscan

https://goerli.etherscan.io/0x39E66855f97011aB7b016f45d744c4cb3f4F012b/

## 8. Lab 6f - Create Crowdsale Token

### Step 1. Create `Crowdsale.sol` in `contracts`

```sol
pragma solidity 0.8.8;
import './TutorialERC20.sol';
import './Ownable.sol';

contract CrowdSale {
```

### Step 2. Create a state variables `_unitPrice`

This is to determine the amount of ETH required to mint 1 ERC20 and `_token` as the token to mint.

```sol
    uint256 _unitPrice;
    TutorialERC20 _token;

    constructor(TutorialERC20 token, uint256 unitPrice)
    {
        _token = token;
        _unitPrice = unitPrice;
    }
```

### Step 3. Create the buy() function.

The buy() function expects an argument for the amount to mint. It is a **payable** function and expects to receive certain amount of ETH from the caller. Before minting the token, it will verify the ETH received matches the price and the amount of tokens to purchase. Finally, it will call the Buy() event if the transaction is successful.

```sol
    event Buy(address buyer, uint256 amount);

    function buy(uint256 amount) public payable{
        require(msg.value >= _unitPrice * amount, "Insufficient payment");
        address buyer = msg.sender;
        _token.mint(buyer, amount);
        emit Buy(buyer, amount);
    }
}
```

### Completed Code

```js
pragma solidity 0.8.8;
import "./TutorialERC20.sol";
import "./Ownable.sol";

contract CrowdSale {
    uint256 _unitPrice;
    TutorialERC20 _token;

    constructor(TutorialERC20 token, uint256 unitPrice) {
        _token = token;
        _unitPrice = unitPrice;
    }

    event Buy(address buyer, uint256 amount);

    function buy(uint256 amount) public payable {
        require(msg.value >= _unitPrice * amount, "Insufficient payment");
        address buyer = msg.sender;
        _token.mint(buyer, amount);
        emit Buy(buyer, amount);
    }
}

```

## 9. Test Crowdsale


### Step 1. Create the `testCrowdsale.js` in `test` directory

```js
const { expect } = require("chai");
const { ethers } = require("hardhat");

describe("Test Crowdsale", () => {
    let accounts;
    let crowdSale;
    let tut;
```

### Step 2. Deploy the TUT and Crowdsale contract

* The Crowdsale contract is deployed with the address of the TUT contract.
* The price is set to 100. That means, each token will cost 100 wei to mint.

**NOTE:** The permission to mint TUT has to be transferred to the Crowdsale contract using `tut.transferOwnership(crowdSale.address);`.

```js
    beforeEach(async () => {
        accounts = await ethers.getSigners();
        let factory = await ethers.getContractFactory("TutorialERC20");
        tut = await factory.deploy("TUT", "TUT", accounts[0].address);
        factory = await ethers.getContractFactory("CrowdSale");
        crowdSale = await factory.deploy(tut.address, 100);
        tut.transferOwnership(crowdSale.address);
    });
```

### Step 3. Create the test `Should sell 100 unit of TUT`.

*To mint 100 unit of TUT, we will need to sent 100 \* 100 wei of ETH along with the call to `buy()` to the Crowdsale contract.

```js
    it("Should sell 100 unit of TUT", async () => {
        const before = await tut.balanceOf(accounts[1].address);
        const response = await crowdSale
            .connect(accounts[1])
            .buy(100, { value: 10000 });
        await response.wait();
        // Assert
        const after = await tut.balanceOf(accounts[1].address);
        expect(before.add(100).toNumber()).equals(after.toNumber());
    });
```

### Complete Code

```js
const { expect } = require("chai");
const { ethers } = require("hardhat");

describe("Test Crowdsale", () => {
    let accounts;
    let crowdSale;
    let tut;

    beforeEach(async () => {
        accounts = await ethers.getSigners();
        let factory = await ethers.getContractFactory("TutorialERC20");
        tut = await factory.deploy("TUT", "TUT", accounts[0].address);
        factory = await ethers.getContractFactory("CrowdSale");
        crowdSale = await factory.deploy(tut.address, 100);
        tut.transferOwnership(crowdSale.address);
    });

    it("Should sell 100 unit of TUT", async () => {
        const before = await tut.balanceOf(accounts[1].address);
        const response = await crowdSale
            .connect(accounts[1])
            .buy(100, { value: 10000 });
        await response.wait();
        // Assert
        const after = await tut.balanceOf(accounts[1].address);
        expect(before.add(100).toNumber()).equals(after.toNumber());
    });
});

```


### Step 4. Run Test

In [ ]:
hh test test/testCrowdsale

## 10. Lab 6g: Verify Contract

https://docs.alchemy.com/alchemy/tutorials/hello-world-smart-contract/submitting-your-smart-contract-to-etherscan
https://hardhat.org/plugins/nomiclabs-hardhat-etherscan#complex-arguments

The key point of having contract readable by everyone is so that it can be validated by anyone. However, since the contracts deployed on the blockchain is in the form of bytecodes, it cannot be interpreted by human. Therefore, etherscan provides a mechanism for the owner of the contract to upload the source code and the source code is verified by etherscan to be associated with the bytecode and the contract on the blockchain. If the source code is successfully verified, anyone viewing the contract on etherscan will see a green tick by the contract box and is able to interact with the contract directly via etherscan.

### Step 1. Register with etherscan

a) Login to etherscan and apply for API Key at https://etherscan.io/myapikey. 
b) Give the API a name **API-NAME**, eg. TRT

### Step 2. Add etherscan API Key to .env

Add the API key to .env as FIN579_ETHERSCAN

```.env
FIN579_MNEMONIC=...
FIN579_ALCHEMY_URLAPIKEY=...
FIN579_ETHERSCAN=...
```

### Step 3. Install Etherscan plugin


In [ ]:
npm i -D @nomiclabs/hardhat-etherscan

### Step 4. Update HardHat config

```js
require("@nomiclabs/hardhat-ethers");
require("@nomiclabs/hardhat-etherscan");
require("dotenv").config();
module.exports = {
    solidity: "0.8.8",
    networks: {
        goerli: {
            chainId: 5,
            url: process.env.FIN579_ALCHEMY_URLAPIKEY,
            accounts: {
                mnemonic: process.env.FIN579_MNEMONIC,
            },
        },
        hardhat: {
            accounts: {
                mnemonic: process.env.FIN579_MNEMONIC,
            },
        },
    },
    etherscan: {
        apiKey: process.env.FIN579_ETHERSCAN,
    },
};

```

### Step 5. Create arguments.js file

```
module.exports=[
    'TUT',
    'TUT',
    '0x85c1A424C326BE10554747dBeC1E79ae465b554e'
]
```

### Step 6. Verify contract

#### Syntax

hh verify --constructor-args arguments.js --network goerli **add-your-contract-address-here** **add-your-api-name-here**


In [ ]:
hh verify --constructor-args arguments.js --network goerli 0x39E66855f97011aB7b016f45d744c4cb3f4F012b TUT

Check https://goerli.etherscan.io/address/0x39E66855f97011aB7b016f45d744c4cb3f4F012b#code